In [2]:
from bokeh.layouts import gridplot
from bokeh.io import show, output_notebook
import hail as hl
hl.init()
output_notebook()


Running on Apache Spark version 3.1.2
SparkUI available at http://Inspiron-ubuntu:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.70-5bb98953a4a7
LOGGING: writing to /home/joyd/Desktop/Hail-on-SGX/client/hail-20210719-2035-0.2.70-5bb98953a4a7.log


Loading BokehJS ...

In [3]:
hl.utils.get_1kg('data/')
mt = hl.read_matrix_table('data/1kg.mt')
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))
mt = mt.annotate_cols(**table[mt.s])
mt = hl.sample_qc(mt)

mt.describe()

2021-07-19 20:36:00 Hail: INFO: 1KG files found
2021-07-19 20:36:02 Hail: INFO: Reading table to impute column types
2021-07-19 20:36:06 Hail: INFO: Finished type imputation
  Loading field 'Sample' as type str (imputed)
  Loading field 'Population' as type str (imputed)
  Loading field 'SuperPopulation' as type str (imputed)
  Loading field 'isFemale' as type bool (imputed)
  Loading field 'PurpleHair' as type bool (imputed)
  Loading field 'CaffeineConsumption' as type int32 (imputed)


----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'Population': str
    'SuperPopulation': str
    'isFemale': bool
    'PurpleHair': bool
    'CaffeineConsumption': int32
    'sample_qc': struct {
        dp_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
        n_transversion: int64, 
        n_star: int64, 
      

In [5]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4)
                    & (mt.sample_qc.call_rate >= 0.97))
ab = mt.AD[1] / hl.sum(mt.AD)
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                       (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                       (mt.GT.is_hom_var() & (ab >= 0.9)))
mt = mt.filter_entries(filter_condition_ab)
mt = hl.variant_qc(mt).cache()
common_mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)
gwas = hl.logistic_regression_rows(test="wald",
                                   y=common_mt.PurpleHair, x=common_mt.GT.n_alt_alleles(), covariates=[1.0, common_mt.isFemale])
pca_eigenvalues, pca_scores, _ = hl.hwe_normalized_pca(common_mt.GT)


2021-07-19 20:37:46 Hail: INFO: logistic_regression_rows: running wald on 250 samples for response variable y,
    with input variable x, and 2 additional covariates...
2021-07-19 20:37:50 Hail: INFO: hwe_normalized_pca: running PCA using 9087 variants.
2021-07-19 20:37:51 Hail: INFO: pca: running PCA with 10 components...


In [6]:
p = hl.plot.qq(gwas.p_value, collect_all=True)
p2 = hl.plot.qq(gwas.p_value, n_divisions=75)

show(gridplot([p, p2], ncols=2, plot_width=400, plot_height=400))


2021-07-19 20:38:06 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-07-19 20:38:09 Hail: INFO: Ordering unsorted dataset with network shuffle


In [7]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

In [9]:
gwas.describe()
gwas.show()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'beta': float64 
    'standard_error': float64 
    'z_stat': float64 
    'p_value': float64 
    'fit': struct {
        n_iterations: int32, 
        converged: bool, 
        exploded: bool
    } 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


+---------------+------------+-----------+----------------+-----------+
| locus         | alleles    |      beta | standard_error |    z_stat |
+---------------+------------+-----------+----------------+-----------+
| locus<GRCh37> | array<str> |   float64 |        float64 |   float64 |
+---------------+------------+-----------+----------------+-----------+
| 1:904165      | ["G","A"]  | -1.79e-01 |       2.57e-01 | -6.95e-01 |
| 1:1563691     | ["T","G"]  | -1.15e-01 |       5.69e-01 | -2.02e-01 |
| 1:1707740     | ["T","G"]  | -1.66e-01 |       2.33e-01 | -7.16e-01 |
| 1:2284195     | ["T","C"]  | -3.82e-02 |       1.86e-01 | -2.05e-01 |
| 1:2779043     | ["T","C"]  | -1.88e-01 |       1.96e-01 | -9.57e-01 |
| 1:2944527     | ["G","A"]  | -7.98e-02 |       2.27e-01 | -3.52e-01 |
| 1:3761547     | ["C","A"]  |  4.50e-01 |       9.21e-01 |  4.88e-01 |
| 1:3803755     | ["T","C"]  | -7.34e-02 |       1.82e-01 | -4.03e-01 |
| 1:4121584     | ["A","G"]  |  4.65e-02 |       1.71e-01 |  2.71e-01 |
| 1:4170048     | ["C","T"]  | -1.09e-01 |       2.06e-01 | -5.32e-01 |
+---------------+------------+-----------+----------------+-----------+

+----------+------------------+---------------+--------------+
|  p_value | fit.n_iterations | fit.converged | fit.exploded |
+----------+------------------+---------------+--------------+
|  float64 |            int32 |          bool |         bool |
+----------+------------------+---------------+--------------+
| 4.87e-01 |                3 |          True |        False |
| 8.40e-01 |                3 |          True |        False |
| 4.74e-01 |                3 |          True |        False |
| 8.38e-01 |                3 |          True |        False |
| 3.39e-01 |                3 |          True |        False |
| 7.25e-01 |                3 |          True |        False |
| 6.26e-01 |                4 |          True |        False |
| 6.87e-01 |                3 |          True |        False |
| 7.86e-01 |                3 |          True |        False |
| 5.95e-01 |                3 |          True |        False |
+----------+------------------+---------------+--------------+
showing top 10 rows

In [10]:
gwas.export("logistic.tsv")

2021-07-19 20:47:06 Hail: INFO: merging 16 files totalling 1.0M...
2021-07-19 20:47:06 Hail: INFO: while writing:
    logistic.tsv
  merge time: 25.323ms


In [12]:
common_mt.PurpleHair.export("PurpleHair.tsv")
common_mt.isFemale.export("isFemale.tsv")

2021-07-19 20:51:23 Hail: INFO: merging 16 files totalling 3.3K...
2021-07-19 20:51:23 Hail: INFO: while writing:
    PurpleHair.tsv
  merge time: 10.785ms
2021-07-19 20:51:23 Hail: INFO: merging 16 files totalling 3.3K...
2021-07-19 20:51:23 Hail: INFO: while writing:
    isFemale.tsv
  merge time: 19.395ms
